In [8]:
import validation_step
import importlib
importlib.reload(validation_step)
from validation_step import *
from etl_eda import *
from sklearn.model_selection import TimeSeriesSplit
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error


In [9]:
data = load('data.csv')

In [37]:
validator = Validator(data)
if validator.validate():
    print("Validation is passed")
else:
    print("Validation is failed")

All values in column item_category_id are numeric
All values in column 'item_category_id' are in range: [0, 83].
All values in column main_category_id are numeric
All values in column 'main_category_id' are in range: [0, 19].
All values in column minor_category_id are numeric
All values in column 'minor_category_id' are in range: [0, 66].
All values in column item_id are numeric
All values in column 'item_id' are in range: [0, 22169].
All values in column shop_id_encoded are numeric
All values in column 'shop_id_encoded' are in range: [0, 57].
All values in column city_id are numeric
All values in column 'city_id' are in range: [0, 29].
All values in column date_block_num are numeric
All values in column 'date_block_num' are in range: [0, 34].
All values in column revenue_log are numeric
All values in column revenue_log are positive
All values in column item_price_log are numeric
All values in column item_price_log are positive
All values in column month are numeric
All values in colum

In [3]:
X_test = data.drop('item_cnt_month_log', axis = 1).loc[data.date_block_num == 34]

In [4]:
model = DummyRegressor(strategy= 'mean')

Train/val split + check if we have enough data

In [5]:
tss = TimeSeriesSplit(n_splits = 3)
n = 0

for train_index, validation_index in tss.split(data.loc[data.date_block_num != 34]):
    train_data, validation_data = data.iloc[train_index], data.iloc[validation_index]
    X_train = train_data.drop('item_cnt_month_log', axis = 1)
    y_train = train_data['item_cnt_month_log']
    X_val = validation_data.drop('item_cnt_month_log', axis = 1)
    y_val = validation_data['item_cnt_month_log']

    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    mse = mean_squared_error(y_pred, y_val)
    n += 1
    print(f'MSE for fold №{n}: {mse}')

MSE for fold №1: 0.035554202977984124
MSE for fold №2: 0.026481845637343997
MSE for fold №3: 0.02788435463394971


The obtained MSE we could use in future as baselines, to ensure that we do not have data leakage